In [2]:
import madpose
#import poselib

import json
import os

import cv2
import numpy as np

import madpose
from madpose.utils import compute_pose_error, get_depths

In [3]:
# 1. Configure RANSAC options
options = madpose.HybridLORansacOptions()
options.min_num_iterations = 100
options.max_num_iterations = 1000
options.success_probability = 0.9999
options.random_seed = 0
options.final_least_squares = True
options.threshold_multiplier = 5.0
# Set reprojection (e.g., 4px) and epipolar error thresholds
options.squared_inlier_thresholds = [4.0**2, 4.0**2] 
options.data_type_weights = [1.0, 1.0]

# 2. Configure Estimator settings
est_config = madpose.EstimatorConfig()
est_config.min_depth_constraint = True # Ensures positive depth
est_config.use_shift = True            # Models depth shift
est_config.ceres_num_threads = 8       # Set to number of physical CPU cores

In [3]:
sample_path = "examples/image_pairs/1_eth3d"
# Read the image pair
image0 = cv2.imread(os.path.join(sample_path, "image0.png"))
image1 = cv2.imread(os.path.join(sample_path, "image1.png"))

# Read info
with open(os.path.join(sample_path, "info.json")) as f:
    info = json.load(f)

# Load camera intrinsics
K0 = np.array(info["K0"])
K1 = np.array(info["K1"])

# Load pre-computed keypoints (you can also run keypoint detectors of your choice)
matches_0_file = os.path.join(sample_path, info["matches_0_file"])
matches_1_file = os.path.join(sample_path, info["matches_1_file"])
mkpts0 = np.load(matches_0_file)
mkpts1 = np.load(matches_1_file)

# Load pre-computed depth maps (you can also run Monocular Depth models of your choice)
depth_0_file = os.path.join(sample_path, info["depth_0_file"])
depth_1_file = os.path.join(sample_path, info["depth_1_file"])
depth_map0 = np.load(depth_0_file)
depth_map1 = np.load(depth_pp1_file)

# Query the depth priors of the keypoints
depth0 = get_depths(image0, depth_map0, mkpts0)
depth1 = get_depths(image1, depth_map1, mkpts1)

# Compute the principal points
pp0 = (np.array(image0.shape[:2][::-1]) - 1) / 2
pp1 = (np.array(image1.shape[:2][::-1]) - 1) / 2
pp = pp0

# Run hybrid estimation
pose, stats = madpose.HybridEstimatePoseScaleOffsetSharedFocal(
    mkpts0,
    mkpts1,
    depth0,
    depth1,
    [depth_map0.min(), depth_map1.min()],
    pp0,
    pp1,
    options,
    est_config,
)
# rotation and translation of the estimated pose
R_est, t_est = pose.R(), pose.t()
# scale and offsets of the affine corrected depth maps
s_est, o0_est, o1_est = pose.scale, pose.offset0, pose.offset1
# estimated shared focal length
f_est = pose.focal

# Load the GT Pose
T_0to1 = np.array(info["T_0to1"])

# Compute the pose error
err_t, err_R = compute_pose_error(T_0to1, R_est, t_est)

# Compute the focal error
f_mean = np.mean([K0[0, 0], K0[1, 1], K1[0, 0], K1[1, 1]])
err_f = np.abs(f_est - f_mean) / f_mean

print("--- Hybrid Estimation Results ---")
print(f"Rotation Error: {err_R:.4f} degrees")
print(f"Translation Error: {err_t:.4f} degrees")
print(f"Focal Error: {(err_f * 100):.2f}%")
print(f"Estimated scale, offset0, offset1: {s_est:.4f}, {o0_est:.4f}, {o1_est:.4f}")

NameError: name 'depth_pp1_file' is not defined

In [ ]:
depth0.shape

In [ ]:
mkpts0

In [4]:
# Try importing the indoor specific model
from romatch import roma_indoor

# Initialize
roma_model = roma_indoor(device="cuda")

2025-12-17 10:09:51.063 | INFO     | romatch.models.model_zoo.roma_models:roma_model:61 - Using coarse resolution (560, 560), and upsample res (864, 864)


In [ ]:
img_path0=os.path.join(sample_path, "image0.png")
img_path1=os.path.join(sample_path, "image1.png")
warp, certainty = roma_model.match(img_path0, img_path1, device="cuda")
# Sample matches for estimation
matches, certainty = roma_model.sample(warp, certainty)
# Convert to pixel coordinates (RoMa produces matches in [-1,1]x[-1,1])

In [ ]:
certainty.shape

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torch

def visualize_roma_matches(imA_path, imB_path, matches, roma_model):
    # 1. Load images to get dimensions
    imA = Image.open(imA_path)
    imB = Image.open(imB_path)
    W_A, H_A = imA.size
    W_B, H_B = imB.size

    # 2. Convert normalized matches [-1, 1] to pixel coordinates
    # RoMa's to_pixel_coordinates handles the math for you
    kptsA, kptsB = roma_model.to_pixel_coordinates(matches, H_A, W_A, H_B, W_B)

    # Move to CPU/Numpy for plotting
    kptsA = kptsA.cpu().numpy()
    kptsB = kptsB.cpu().numpy()

    # 3. Create the plot
    fig, ax = plt.subplots(figsize=(20, 10))
    
    # Concatenate images side-by-side
    imA_np = np.array(imA)
    imB_np = np.array(imB)
    
    # Handle different heights by padding (optional, but good for robustness)
    hA, wA, c = imA_np.shape
    hB, wB, c = imB_np.shape
    max_h = max(hA, hB)
    
    # Create canvas
    canvas = np.zeros((max_h, wA + wB, 3), dtype=np.uint8)
    canvas[:hA, :wA, :] = imA_np
    canvas[:hB, wA:wA+wB, :] = imB_np
    
    ax.imshow(canvas)
    ax.axis('off')

    # 4. Draw matches
    # Shift kptsB x-coordinates by width of image A
    kptsB_shifted = kptsB.copy()
    kptsB_shifted[:, 0] += wA

    # Plot lines
    # Using a random subset if there are too many matches to see clearly
    num_to_plot = min(len(kptsA), 500) 
    indices = np.random.choice(len(kptsA), num_to_plot, replace=False)

    for i in indices:
        # Draw line
        ax.plot([kptsA[i, 0], kptsB_shifted[i, 0]], 
                [kptsA[i, 1], kptsB_shifted[i, 1]], 
                c='lime', linewidth=0.5, alpha=0.7)
        
        # Draw dots
        ax.scatter(kptsA[i, 0], kptsA[i, 1], c='lime', s=5)
        ax.scatter(kptsB_shifted[i, 0], kptsB_shifted[i, 1], c='lime', s=5)

    plt.tight_layout()
    plt.show()

# --- USAGE ---
# Assuming you already ran:
# matches, certainty = roma_model.sample(warp, certainty)

visualize_roma_matches(img_path0, img_path1, matches, roma_model)

In [ ]:
imA = Image.open(img_path0)
imB = Image.open(img_path1)
W_A, H_A = imA.size
W_B, H_B = imB.size

# 2. Convert normalized matches [-1, 1] to pixel coordinates
# RoMa's to_pixel_coordinates handles the math for you
kptsA, kptsB = roma_model.to_pixel_coordinates(matches, H_A, W_A, H_B, W_B)

# Move to CPU/Numpy for plotting
kptsA = kptsA.cpu().numpy()
kptsB = kptsB.cpu().numpy()

In [ ]:
kptsA

In [ ]:
pp